<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/Hyperparameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras-tuner==1.0.0

     |████████████████████████████████| 92kB 3.5MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=316966825cab3dd2b4d5e1243975b8f54fbff3327ce71a0a0a638d6b8de71968
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout

In [ ]:
classes = len(lb.classes_)

In [ ]:
HP_img_dims = (96,96,1)
def build(hp):
    classes = len(lb.classes_)
    model = keras.models.Sequential()
    # BLOCK1
    model.add(Conv2D(hp.Int('input_units',32,512,32),(3,3), 
                     input_shape=trainx.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout',[0.25,0.4,0.5])))
    

    for i in range(hp.Int("n_layers",1,6)):
      model.add(Conv2D(hp.Int(f'conv_{i}_units',32,512,32),(3,3)))
      model.add(Activation('relu'))
      model.add(Dropout(hp.Choice(f'dropout_{i}',[0.25,0.4,0.5])))

     
   
   

  

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(classes))
    model.add(Activation("softmax"))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
    

    

    return model
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)





In [ ]:
tuner=RandomSearch(
    build,objective='val_accuracy',
    max_trials=15,executions_per_trial=1)

tuner.search(x=trainx,
             y=trainy,
             epochs=5,
             batch_size=64,
             validation_data=(testx,testy))